<h1> Webscraping de um site de torrefadora e tratamento dos dados como Data Frames usando Pandas <h1/>

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import matplotlib.pyplot as plt

In [2]:

shop_response = requests.get("https://manhattancoffeeroasters.com/shop")

shop = bs(shop_response.content, "html.parser")
coffee_links = shop.find(attrs = {"class" : "products columns-3"})
coffee_links2 = coffee_links.find_all("a")

links = []

for link in coffee_links2:
    links.append(link["href"])

links = [links[x] for x in range(0,len(links),2) ]


                               Scraping das páginas individuais dos cafés

                                          DEMORA PARA RODAR
                                              ||||||||||  
                                              VVVVVVVVVV

In [3]:
dict_coffees = {}
price = []
name = []
notes = []
origin = []

for link in links:
    #Request das páginas individuais dos cafés
    coffee_page_response = requests.get(link)
    coffee_page = bs(coffee_page_response.content,"html.parser")
    #Coletando nome dos cafés
    name.append(coffee_page.find_all("h1")[0].get_text())

    #Coletando preço
    price.append(float(coffee_page.select(".price")[0].get_text()[1:6].strip()))
    
    #Coletando notas
    notes.append(coffee_page.find(attrs = {"class":"origin_taste_processing"}).find_all("ul")[1].find_all("li")[1].get_text().strip("\n").strip() + ", " + coffee_page.find(attrs = {"class":"origin_taste_processing"}).find_all("ul")[1].find_all("li")[2].get_text().strip("\n").strip() + ", " + coffee_page.find(attrs = {"class":"origin_taste_processing"}).find_all("ul")[1].find_all("li")[3].get_text().strip("\n").strip())

    #Coletando origem
    origin.append(coffee_page.find(attrs = {"class":"origin_taste_processing"}).find_all("ul")[0].find_all("li")[1].get_text().strip("\n").strip() + ", " + coffee_page.find(attrs = {"class":"origin_taste_processing"}).find_all("ul")[0].find_all("li")[2].get_text().strip("\n").strip())

    dict_coffees[name[-1]] = price[-1],notes[-1],origin[-1]

In [4]:


coffee_df = pd.DataFrame(dict_coffees).transpose()

coffee_df.columns = ['Preco','Notas','Origem']

coffee_df

,Preco,Notas,Origem
Connoisseurs Choice (Espresso),9,"Cherry, Chocolate, Honey","Brazil, Panama"
Diego Bermudez (Filter),13.5,"Lychee, Raspberry, turkish delight","cauca, Colombia"
Erasmus (Espresso),9,"Candied Pecan, Creamy, Plum Jam","Brazil, Panama"
Inacio Soares Catuai (Espresso),8,"Chocolate, praline, White flowers","Brazil, mata de minas"
Inacio Soares Catuai (Filter),8,"Chocolate, praline, White flowers","Brazil, mata de minas"
Kinini (Espresso),9,"Baked Orange, Raspberry, Red Fruit","Rwanda, Mbogo, Rusiga"
Kinini (Filter),9,"Honey, Raspberry, Rhubarb","Rwanda, Mbogo, Rusiga"
Kinini natural(Filter),9,"blackberry, Rhubarb, stewed orange","Rwanda, Mbogo, Rusiga"
Shantawene washed (Filter),13.5,"Blueberry, Juicy, Purple fruit","Ethiopia, bombe mountain"
Sweet Valley (Filter),10.5,"bubblegum, grape, Red apple","Colombia, Valle del Cauca"
